[View in Colaboratory](https://colab.research.google.com/github/juzkev/A-Bridged/blob/master/Repo.ipynb)

In [1]:
# Time
import time
start_t = time.time()

# Google Drive
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#textract
!apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libpulse-dev
!pip install textract
import textract

# DS essentials
import pandas as pd
import numpy as np

# NLP
import re



Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
python-dev is already the newest version (2.7.14-2ubuntu1).
libxml2-dev is already the newest version (2.9.4+dfsg1-4ubuntu1.2).
The following additional packages will be installed:
  dbus fontconfig fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  i965-va-driver libaacs0 libapparmor1 libasound2 libasound2-data libass9
  libasyncns0 libavahi-client3 libavahi-common-data libavahi-common3
  libavc1394-0 libavcodec57 libavdevice57 libavfilter6 libavformat57
  libavresample3 libavutil55 libbdplus0 libbluray2 libboost-filesystem1.62.0
  libboost-system1.62.0 libbs2b0 libcaca0 libcairo2 libcapnp-0.5.3
  libcdio-cdda1 libcdio-paranoia1 libcdio13 libchromaprint1 libcrystalhd3
  libcups2 libcupsfilters1 libcupsimage2 libdatrie1 libdbus-1-3 libdc1394-22
  libdrm-amdgpu1 libdrm-common libdrm-intel1 

In [39]:
# Install Spacy required for fastai.text
!pip install -U spacy
!python -m spacy download en

    100% |████████████████████████████████| 22.0MB 1.0MB/s 
    100% |████████████████████████████████| 122kB 25.2MB/s 
    100% |████████████████████████████████| 1.2MB 11.3MB/s 
    100% |████████████████████████████████| 194kB 24.0MB/s 
    100% |████████████████████████████████| 153kB 24.2MB/s 
    100% |████████████████████████████████| 604kB 17.3MB/s 
    100% |████████████████████████████████| 317kB 19.1MB/s 
    100% |████████████████████████████████| 450kB 21.5MB/s 
    100% |████████████████████████████████| 51kB 18.3MB/s 
    100% |████████████████████████████████| 143kB 20.9MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

In [0]:
import spacy

# Downloading PDFs from Google Drive

In [7]:
    for f in file_list:
      fname = os.path.join(local_download_path, f['title'])
      print('downloading to {}'.format(fname))
      try:
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
      except:pass

downloading to /root/data/SR/2016CMMT CSR report en.pdf
downloading to /root/data/SR/44 Final Sustainability Report to CoSec 10Feb16.pdf
downloading to /root/data/SR/17_0080 Cambridge SR2016_v5single_Final.pdf
downloading to /root/data/SR/Wilmar Sustainability Report 2017.pdf
downloading to /root/data/SR/Wilmar SR 2016 - Final.pdf
downloading to /root/data/SR/Wilmar Policy Progress Report - Dec 2013 to Dec 2015.pdf
downloading to /root/data/SR/Weiye Sustainability Report FY2017_ENG.pdf
downloading to /root/data/SR/VIT 2017 Sustainability Report.pdf
downloading to /root/data/SR/Venture Sustainability Report 2017.pdf
downloading to /root/data/SR/Valuetronics Holdings Limited_Sustainability Report 2018.pdf
downloading to /root/data/SR/ValueMax_20180531 -sustainability report.pdf
downloading to /root/data/SR/UOL_Sustainability_Report_2017.pdf
downloading to /root/data/SR/TMGL Sustainability Report 2017.pdf
downloading to /root/data/SR/Tiong Seng Sustainability Report 2017.pdf
downloading t

# Reading the PDF into DF

In [8]:
filename = '2305_CDL.pdf'
print(os.path.join(local_download_path, filename))
t = textract.process(os.path.join(local_download_path, filename))

/root/data/SR/2305_CDL.pdf


In [24]:
# Reading the PDF
df = pd.DataFrame()
succ = 0
fail = 0
# pattern = 'SDG (\d{2})'
s_t = time.time()

for filename in os.listdir(os.path.join(local_download_path)):
  try:
    print(os.path.join(local_download_path, filename))

    # Textract code
    t = textract.process(os.path.join(local_download_path, filename))

    df=df.append({'filename': filename, 'fulltext': t}, ignore_index=True)
    succ += 1
  except Exception as e: 
    print(e)
    fail += 1

print(f'''Successfully processed {succ}/{succ+fail} pdfs''')
print(f'Reading PDFs took {time.time()-s_t} seconds')

/root/data/SR/Sustainability Report 2016.pdf
/root/data/SR/First Resources Limited - Sustainability Report 2017.pdf
/root/data/SR/Wilmar Policy Progress Report - Dec 2013 to Dec 2015.pdf
/root/data/SR/GAR26-29-06-2018-SR2017.pdf
/root/data/SR/Sanli - Sustainability Report 2018.pdf
/root/data/SR/2017 CMMT CSR report.pdf
/root/data/SR/Wilmar - Second Quarter 2015 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/SR/SR Report_FY2017.pdf
/root/data/SR/Wilmar- Third Quarter 2016 Update Report on its No Deforestation No Peat No Exploitation Policy.pdf
/root/data/SR/Brook Crompton Sustainability Report.pdf
/root/data/SR/Sustainability Report 2018.pdf
/root/data/SR/CDG SR 2015.pdf
/root/data/SR/2017 Sustainability Report.pdf
/root/data/SR/DFI - Sustainability Report 2018.pdf
/root/data/SR/KCL_SR2016.pdf
/root/data/SR/Samko - Annt - Sustainability Report - FY2017 -  Final.pdf
/root/data/SR/Interra SR Report For Upload FINAL.pdf
/root/data/SR/Accordia Golf Trust

In [35]:
df.head()

,filename,fulltext
0,Sustainability Report 2016.pdf,b'The Road to a\n\nSustainable Future\n\nSMRT ...
1,First Resources Limited - Sustainability Repor...,"b""SUSTAINABILITY\nREPORT\n\n2017\nFIRST\nRESOU..."
2,Wilmar Policy Progress Report - Dec 2013 to De...,"b""NO DEFORESTATION,\nNO PEAT,\nNO EXPLOITATION..."
3,GAR26-29-06-2018-SR2017.pdf,b'Changing for the better\n\nSharing responsib...
4,Sanli - Sustainability Report 2018.pdf,"b""\x0cTABLE OF CONTENTS\n02 MESSAGE FROM OUR B..."


In [0]:
doc.sents

In [44]:
nlp = spacy.load('en')
doc = nlp(df.iloc[0,1].decode())
for sent in doc.sents:
    print(sent.text)

The Road to a

Sustainable Future

SMRT Corporation Ltd
Sustainability Report 2016

SMRT Corporation Ltd (SMRT) is Singapore’s
premier multi-modal land transport provider.
Our
core businesses are in rail operations, maintenance
and engineering as well as in bus, taxi and
automotive services.

Complementing these are our integrated
businesses in retail, media and marketing, as
well as properties and retail management.
We
are committed to sustainable development and
corporate social responsibility.

SMRT was established in 1987 and was listed on the
Singapore Exchange in July 2000.
As of 31 March
2016, our market capitalisation stood at around
$2.3 billion.


About this Report
SMRT’s inaugural Sustainability Report 2016 provides an
overview of our approach, priorities and progress in the
area of sustainability for the Financial Year 2016 (1 April
2015 to 31 March 2016).
Sustainability is integral to our
business and as we embark on this journey of sustainability
reporting, we also hope 

In [60]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
tokenizer = PunktSentenceTokenizer()
# tokenizer.train(df.fulltext.st
r.decode('utf-8'))
# tokenizer.tokenize(df.iloc[0,1].decode()

['The Road to a\n\nSustainable Future\n\nSMRT Corporation Ltd\nSustainability Report 2016\n\n\x0cSMRT Corporation Ltd (SMRT) is Singapore’s\npremier multi-modal land transport provider.',
 'Our\ncore businesses are in rail operations, maintenance\nand engineering as well as in bus, taxi and\nautomotive services.',
 'Complementing these are our integrated\nbusinesses in retail, media and marketing, as\nwell as properties and retail management.',
 'We\nare committed to sustainable development and\ncorporate social responsibility.',
 'SMRT was established in 1987 and was listed on the\nSingapore Exchange in July 2000.',
 'As of 31 March\n2016, our market capitalisation stood at around\n$2.3 billion.',
 'About this Report\nSMRT’s inaugural Sustainability Report 2016 provides an\noverview of our approach, priorities and progress in the\narea of sustainability for the Financial Year 2016 (1 April\n2015 to 31 March 2016).',
 'Sustainability is integral to our\nbusiness and as we embark on thi

In [38]:
print(df.iloc[0,1].decode())

The Road to a

Sustainable Future

SMRT Corporation Ltd
Sustainability Report 2016

SMRT Corporation Ltd (SMRT) is Singapore’s
premier multi-modal land transport provider. Our
core businesses are in rail operations, maintenance
and engineering as well as in bus, taxi and
automotive services.
Complementing these are our integrated
businesses in retail, media and marketing, as
well as properties and retail management. We
are committed to sustainable development and
corporate social responsibility.
SMRT was established in 1987 and was listed on the
Singapore Exchange in July 2000. As of 31 March
2016, our market capitalisation stood at around
$2.3 billion.

About this Report
SMRT’s inaugural Sustainability Report 2016 provides an
overview of our approach, priorities and progress in the
area of sustainability for the Financial Year 2016 (1 April
2015 to 31 March 2016). Sustainability is integral to our
business and as we embark on this journey of sustainability
reporting, we also hope to 